In [64]:
###입력해주세요
era = '90년대'

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import re
from selenium.common.exceptions import NoSuchElementException
import csv
import pandas as pd
from urllib.request import urlopen
import sys
import io
import glob
import os

In [70]:
#기존 csv 파일 읽어들이기
path = r'../crawler/90년대_파일이름변경'                     # use your path
all_files = glob.glob(os.path.join(path, "*.csv")) 

results = []
attr_names = ['rank', 'sond_id', 'title', 'singers', 'album', 'date','genre', 'creator', 'lyric']


for f in range(len(all_files)):
    if f % 2 == 0:
        count = 0
        with open(str(all_files[f]), newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if count < 10:
                    result = {}
                    result['rank'] = row['rank']
                    result['song_id'] = row['song_id']
                    result['title'] = row['title']
                    result['singers'] = row['singers']
                    result['album'] = row['album']
                    result['date'] = row['date']
                    result['genre'] = row['genre']
                    result['creator'] = row['creator']
                    result['lyric'] = row['lyric']    
                    results.append(result)
                    count += 1
                else:
                    break

In [36]:
max_l = 0
for result in results:
    if max_l < len(result['lyric']):
        max_l = len(result['lyric'])

In [37]:
print(max_l)

2660


In [8]:
count = 0
for result in results:
    count += 1
    if result['song_id'] == 621079:
        break
print(count)

2610


In [7]:
len(results)

2610

In [21]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(2)
singer_ids = []
song_sings = []
song_ids = []
addtional_results = []
count = 0

#singer_id, debut, type, gender, company
for i in range(270,len(results)):
    count += 1
    print("--------------------start-------------------------")
    song_sing = {}
    song_id = results[i]['song_id']
    
    if song_id in song_ids:
        pass
    else:
        song_ids.append(song_id)

        song_sing['song_id'] = song_id
        print("song_id : " + song_id)
        driver.get('https://www.melon.com/song/lyrics.htm?songId='+song_id)
        driver.implicitly_wait(5)

        additional_result = {}
        #additional_result['song_id'] = song_id

        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")

        #불러오지 않은 정보라면

        singer_chunk = soup.select('#downloadfrm > div > div > div.entry > div.info > div.artist > a')[0]
        singer_id = str(singer_chunk).split('goArtistDetail')[1].split('"')[0].split("'")[1]
        additional_result['singer_id'] = singer_id  
        print("singer_id : " + singer_id)
        song_sing['singer_id'] = singer_id
        song_sings.append(song_sing)


        if singer_id not in singer_ids:
            singer_ids.append(singer_id)
            driver.implicitly_wait(5)
            singer_search = driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[2]/a')
            singer_search.click()
            driver.implicitly_wait(2)
            #print("singer_search")
            detail_page = driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[2]/a')
            detail_page.click()
            
            driver.implicitly_wait(5)
            #print("detail_page")

            html = driver.page_source
            soup = BeautifulSoup(html, "lxml")
            driver.implicitly_wait(2)            
            #데뷔, 타입, 소속사명 있는 블록
            
            debut_num = 0
            type_num = 0
            company_num = 0
            #### error
            try :
                block = soup.select('#conts > div.section_atistinfo03 > dl')[0]
                block = str(block).replace('\n','')
                block = str(block).replace('\t','')
                array = str(block).split('<dt>')



                for a in range(len(array)):
                    if '데뷔' in array[a]:
                        debut_num = a
                    elif '유형' in array[a]:
                        type_num = a
                    elif '소속사명' in array[a]:
                        company_num = a
            except:
                pass

            if debut_num != 0:
                debut = soup.select('#conts > div.section_atistinfo03 > dl > dd:nth-of-type({})'.format(debut_num))[0].text
                #print(debut)
                if '|' in debut:
                    debut = debut.split('|')[0].replace('\n','')
                    print("debut : "+debut)
                additional_result['debut'] = debut

            if type_num != 0:
                detail = soup.select('#conts > div.section_atistinfo03 > dl > dd:nth-of-type({})'.format(type_num))[0].text
                detail = detail.replace('\n','')
                detail = detail.replace('\t','')
                detail = detail.split('|')

                singer_type = detail[0]
                singer_gender = detail[1]
                print("singer_type : "+singer_type)
                print("singer_gender : "+singer_gender)

                additional_result['singer_type'] = singer_type
                additional_result['singer_gender'] = singer_gender

            if company_num != 0:
                company = soup.select('#conts > div.section_atistinfo03 > dl > dd:nth-of-type({})'.format(company_num))[0].text
                print("company : "+company)
                additional_result['company'] = company
                
            html = driver.page_source
            soup = BeautifulSoup(html, "lxml") 
            
            try:
                block2 = soup.select('#conts > div.section_atistinfo04')[0]
                block2 = str(block2).replace('\n','')
                block2 = str(block2).replace('\t','')
                array2 = str(block2).split('<dt>')

                bth_num = 0
                for a in range(len(array2)):
                    if '생일' in array2[a]:
                        bth_num = a
                if bth_num != 0:
                    bthday = soup.select('#conts > div.section_atistinfo04 > dl > dd:nth-of-type({})'.format(bth_num))[0].text
                    additional_result['bthday'] = bthday
                    print("birthday : "+bthday)
            except:
                pass

            addtional_results.append(additional_result)
        else:
            print("이미 가져온 가수 정보")
            pass
print(count)

--------------------start-------------------------
song_id : 10434
singer_id : 100007
singer_type : 그룹
singer_gender : 남성
company : 아이에이치큐
--------------------start-------------------------
song_id : 89806
singer_id : 100177
singer_type : 솔로
singer_gender : 남성
company : 주식회사 신엔터테인먼트
--------------------start-------------------------
song_id : 9040
singer_id : 983
singer_type : 솔로
singer_gender : 여성
company : 세이렌엔터테인먼트
birthday : 1969.12.29
--------------------start-------------------------
song_id : 105639
singer_id : 100016
singer_type : 그룹
singer_gender : 여성
company : (주)SM엔터테인먼트
--------------------start-------------------------
song_id : 9419
singer_id : 1081
singer_type : 솔로
singer_gender : 남성
birthday : 1965.08.14
--------------------start-------------------------
song_id : 10291
singer_id : 1142
singer_type : 솔로
singer_gender : 남성
company : (주)앤에이취이엠쥐
birthday : 1973.11.30
--------------------start-------------------------
song_id : 489863
singer_id : 1008
singer_type : 솔로
singe

singer_id : 100014
singer_type : 솔로
singer_gender : 남성
company : 안테나뮤직
birthday : 1971.04.19
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
song_id : 104868
singer_id : 5675
debut : 1991.06
singer_type : 솔로
singer_gender : 여성
company : 캔 엔터테인먼트
birthday : 1965.05.15
--------------------start-------------------------
song_id : 90434
singer_id : 100185
singer_type : 그룹
singer_gender 

singer_id : 1176
singer_type : 솔로
singer_gender : 남성
company : H.W 엔터프라이즈
birthday : 1966.03.06
--------------------start-------------------------
song_id : 104419
singer_id : 100252
singer_type : 그룹
singer_gender : 남성
--------------------start-------------------------
--------------------start-------------------------
song_id : 503258
singer_id : 2076
singer_type : 솔로
singer_gender : 여성
company : RocKing 엔터테인먼트
birthday : 1978.03.03
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
song_id : 111209
singer_id : 925
singer_type : 솔로
singer_gender : 남성
company : 뮤직팜
birthday : 1974
--------------------start-------------------------
--------------------start-------------------------
song_id : 91823
singer_id : 1100
debut : 1993.07
singer_type : 솔로
singer_gender : 여성
company : 에이팝엔터테인먼트
birthday : 1969.08.17
--------------------start-----

singer_id : 1424
singer_type : 솔로
singer_gender : 남성
company : 후너스 크리에이티브 
birthday : 1969.06.14
--------------------start-------------------------
--------------------start-------------------------
song_id : 502946
singer_id : 1790
이미 가져온 가수 정보
--------------------start-------------------------
song_id : 517708
singer_id : 25
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
song_id : 489210
singer_id : 100195
singer_type : 그룹
singer_gender : 혼성
company : 티엔터테인먼트
--------------------start-------------------------
--------------------start-------------------------
song_id : 482404
singer_id : 102409
singer_type : 그룹
singer_gender : 남성
company : 주식회사 엘리야엔터테인먼트
--------------------start-------------------------
--------------------start-------------------------
------------

singer_id : 687
singer_type : 솔로
singer_gender : 남성
birthday : 1964.09.17
--------------------start-------------------------
song_id : 431618
singer_id : 17671
singer_type : 솔로
singer_gender : 남성
company : (주) 레인컴퍼니
birthday : 1982.06.25
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
song_id : 111414
singer_id : 10577
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
song_id : 430125
singer_id : 1582
이미 가져온 가수 정보
--------------------start-------------------------
song_id : 430051
singer_id : 101491
singer_type : 그룹
singer_gender : 혼성
company : 뮤직팜
--------------------start-------------------------
song_id : 317991
singer_id : 891
이미 가져온 가수 정보
--------------------start

...

singer_id : 198094
이미 가져온 가수 정보
--------------------start-------------------------
song_id : 1772218
singer_id : 35007
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
song_id : 1788174
singer_id : 181694
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
song_id : 1789551
singer_id : 232696
singer_type : 솔로
singer_gender : 여성
company : 울림엔터테인먼트
birthday : 1990.10.11
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
song_id : 1789828
singer_id : 237650
singer_type : 솔로
singer_gender : 남성
birthday : 1985.02.27
--------------------start-------------------------
--------------------start-----------------------

singer_id : 247639
singer_type : 그룹
singer_gender : 남성
company : (주)SM엔터테인먼트
--------------------start-------------------------
--------------------start-------------------------
song_id : 1884329
singer_id : 3305
이미 가져온 가수 정보
--------------------start-------------------------
song_id : 1897011
singer_id : 236815
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
song_id : 1897284
singer_id : 1100
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
song_id : 1903050
singer_id : 1713
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
---------------

singer_id : 108581
이미 가져온 가수 정보
--------------------start-------------------------
song_id : 2012282
singer_id : 228069
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
song_id : 1951552
singer_id : 403841
singer_type : 솔로
singer_gender : 남성
company : (주)YG엔터테인먼트
birthday : 1990.12.12
--------------------start-------------------------
--------------------start-------------------------
song_id : 1984049
singer_id : 222128
singer_type : 그룹
singer_gender : 여성
company : DSP미디어
--------------------start-------------------------
song_id : 1996934
singer_id : 57528
singer_type : 솔로
singer_gender : 남성
company : J-SOUL company
birthday : 1983.12.04
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
song_id : 1964754
singer_id : 108951
이미 가져온 가수 정보
--------------------start-------

singer_id : 417791
singer_type : 솔로
singer_gender : 남성
company : Stone Music Entertainment, B2M Entertainment
birthday : 1984.09.12
--------------------start-------------------------
song_id : 2172570
singer_id : 247639
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
song_id : 2203097
singer_id : 228069
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
song_id : 2197448
singer_id : 417985
singer_type : 그룹
singer_gender : 여성
company : 큐브엔터테인먼트
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
song_id : 2135470
singer_id : 259822
이미 가져온 가수 정보
--------------------start-------------------------
song_id : 2200798
singer_id : 58793
이미 가져온 가수 정보
--------------------start-------------------------
----------------

singer_id : 175139
이미 가져온 가수 정보
--------------------start-------------------------
song_id : 2343748
singer_id : 259822
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
song_id : 2345593
singer_id : 428501
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
song_id : 2366796
singer_id : 322666
이미 가져온 가수 정보
--------------------start-------------------------
--------------------start-------------------------
song_id : 2361466
singer_id : 441844
singer_type : 솔로
singer_gender : 여성
company : YG 엔터테인먼트
birthday : 1991.02.26
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
--------------------start-------------------------
song_id : 2349352
singer_id : 259928
singer_type : 그룹
singer_gender : 남성
company : (주)앤에이취이엠쥐
--------------------start-----------------------

In [18]:
with open('./song_singer_match_{}.csv'.format(era), 'w') as csvfile:
    fieldnames = ['song_id', 'singer_id']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(song_sings)):
        writer.writerow(song_sings[i])

['30', '6', '19', '4788', '990', '859', '23', '102421', '32', '100751', '812', '751', '1012', '914', '5023', '2020', '14065', '448', '1169', '5570', '7697', '33', '4826', '29', '40', '102013', '101012', '966', '7684', '5722', '101971', '21', '1062', '1004', '5761', '13078', '27', '12', '101013', '5856', '6179']


In [35]:
print(len(song_sings))

729


In [12]:
print(count)

170


In [22]:
with open('./additional_result_{}.csv'.format(era), 'a') as csvfile:
    fieldnames = ['singer_id', 'debut', 'singer_type', 'singer_gender', 'company', 'bthday']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(addtional_results)):
        writer.writerow(addtional_results[i])

In [21]:

print(addtional_results[10])

{'song_id': '1005216', 'singer_id': '812', 'debut': '1978', 'singer_type': '솔로', 'singer_gender': '남성'}


In [29]:
modifieds = []
with open('./additional_result_00년대_modified.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        modified = {}
        modified['singer_id'] = row['singer_id']
        modified['debut'] = row['debut']
        modified['singer_type'] = row['singer_type']
        modified['singer_gender'] = row['singer_gender']
        modified['company'] = row['company']
        modified['bthday'] = row['bthday']
        modifieds.append(modified)

In [31]:
unique_modifieds = []
unique_id = []
for modified in modifieds:
    if modified['singer_id'] not in unique_id:
        unique_id.append(modified['singer_id'])
        unique_modifieds.append(modified)

In [33]:
print(len(modifieds))

288


In [34]:
with open('./additional_result_{}_final.csv'.format(era), 'w') as csvfile:
    fieldnames = ['singer_id', 'debut', 'singer_type', 'singer_gender', 'company', 'bthday']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(unique_modifieds)):
        writer.writerow(unique_modifieds[i])

In [47]:
song_ids = []
unique_result = []
for result in results:
    song_id = result['song_id']
    if song_id not in song_ids:
        song_ids.append(song_id)
        unique_result.append(result)

In [48]:
with open('./concated_result_{}.csv'.format(era), 'w') as csvfile:
    fieldnames = ['rank', 'song_id','title', 'singers', 'album', 'date', 'genre', 'creator', 'lyric']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(unique_result)):
        writer.writerow(unique_result[i])

In [41]:
singer_songs = []
with open('./song_singer_match_00년대.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        singer_song = {}
        singer_song['song_id'] = row['song_id']
        singer_song['singer_id'] = row['singer_id']
        singer_songs.append(singer_song)

In [44]:
uniques = []
unique_id = []
for singer_song in singer_songs:
    if singer_song['song_id'] not in unique_id:
        unique_id.append(singer_song['singer_id'])
        uniques.append(singer_song)

In [45]:
with open('./song_singer_match_00년대_unique.csv', 'w') as csvfile:
    fieldnames = ['song_id', 'singer_id']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(uniques)):
        writer.writerow(uniques[i])

In [46]:
print(uniques)

[{'song_id': '38541', 'singer_id': '889'}, {'song_id': '489866', 'singer_id': '1008'}, {'song_id': '91324', 'singer_id': '100007'}, {'song_id': '1139376', 'singer_id': '969'}, {'song_id': '105690', 'singer_id': '100065'}, {'song_id': '85257', 'singer_id': '3895'}, {'song_id': '37992', 'singer_id': '1081'}, {'song_id': '91320', 'singer_id': '100022'}, {'song_id': '422691', 'singer_id': '969'}, {'song_id': '85844', 'singer_id': '100177'}, {'song_id': '85674', 'singer_id': '100030'}, {'song_id': '3105064', 'singer_id': '100997'}, {'song_id': '1564057', 'singer_id': '995'}, {'song_id': '86166', 'singer_id': '100183'}, {'song_id': '83192', 'singer_id': '100016'}, {'song_id': '489869', 'singer_id': '1008'}, {'song_id': '85258', 'singer_id': '3895'}, {'song_id': '9879', 'singer_id': '4326'}, {'song_id': '607552', 'singer_id': '969'}, {'song_id': '1606631', 'singer_id': '100059'}, {'song_id': '85900', 'singer_id': '1790'}, {'song_id': '91325', 'singer_id': '100007'}, {'song_id': '86668', 'sing

In [71]:
print(len(results))

2580


In [72]:
song_ids = []
unique_result = []
for result in results:
    song_id = result['song_id']
    if song_id not in song_ids:
        song_ids.append(song_id)
        unique_result.append(result)

In [73]:
with open('./concated_result_{}.csv'.format(era), 'w') as csvfile:
    fieldnames = ['rank', 'song_id','title', 'singers', 'album', 'date', 'genre', 'creator', 'lyric']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(unique_result)):
        writer.writerow(unique_result[i])